In [1]:
path = 'titles.csv'

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sentence_transformers import SentenceTransformer, util



In [5]:
x = pd.read_csv(path)
x.head()

,title
0,Solving QBF Instances with Nested SAT Solvers.
1,Multiscale Manifold Learning.
2,Online Transfer Learning for Differential Diag...
3,Logic Programming in Assumption-Based Argument...
4,Role-Aware Conformity Modeling and Analysis in...


In [6]:
x = x[x['title'].str.split().str.len() > 10]        # remove short titles (less than 10 words)

x['clean'] = x['title'].str.replace('\d+', '')
x['clean'] = x['clean'].str.replace('[^\w\s]', '')
x['clean'] = x['clean'].str.lower()
x.tail()


,title,clean
119441,A Case Study on the Use of Semantic Web Techno...,a case study on the use of semantic web techno...
119444,An Application Framework for Delivering Multim...,an application framework for delivering multim...
119445,Newspaper editors vs the crowd: on the appropr...,newspaper editors vs the crowd on the appropri...
119448,Patch-based skin color detection and its appli...,patchbased skin color detection and its applic...
119449,Detecting image spam using local invariant fea...,detecting image spam using local invariant fea...


In [8]:
# reset index
x = x.reset_index(drop=True)
x.tail()

,title,clean
24464,A Case Study on the Use of Semantic Web Techno...,a case study on the use of semantic web techno...
24465,An Application Framework for Delivering Multim...,an application framework for delivering multim...
24466,Newspaper editors vs the crowd: on the appropr...,newspaper editors vs the crowd on the appropri...
24467,Patch-based skin color detection and its appli...,patchbased skin color detection and its applic...
24468,Detecting image spam using local invariant fea...,detecting image spam using local invariant fea...


In [9]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
# print moel parameters
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)


In [10]:
# add embeddings to dataframe
x['embeddings'] = x['clean'].apply(lambda x: model.encode(x))

In [11]:
x.head()

,title,clean,embeddings
0,Logic Programming in Assumption-Based Argument...,logic programming in assumptionbased argumenta...,"[-0.023969512, 0.010221153, 0.0005422938, -0.0..."
1,Modeling and Mining Spatiotemporal Patterns of...,modeling and mining spatiotemporal patterns of...,"[0.0034898794, -0.0044532777, -0.053476732, -0..."
2,Mitosis Detection in Breast Cancer Histology I...,mitosis detection in breast cancer histology i...,"[0.013757976, -0.00069692935, 0.022236006, -0...."
3,Towards Population Scale Activity Recognition:...,towards population scale activity recognition ...,"[0.06615189, 0.014459885, -0.035167716, -0.081..."
4,A Platform to Evaluate the Technology for Serv...,a platform to evaluate the technology for serv...,"[-0.0020297964, -0.03787091, -0.046355557, -0...."


In [12]:
x.to_csv('titles_embeddings.csv', index=False)

In [17]:
titles = x['title'].tolist()
embeddings = x['embeddings'].tolist()

# save titles and embeddings to disk
import pickle
with open('titles.pkl', 'wb') as f:
    pickle.dump(titles, f)

with open('embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [19]:
# load titles and embeddings from disk 
with open('titles.pkl', 'rb') as f:
    ltitles = pickle.load(f)
    print(type(titles))
    print(len(titles))

with open('embeddings.pkl', 'rb') as f:
    lembeddings = pickle.load(f)
    print(type(embeddings))
    print(len(embeddings))

<class 'list'>
24469
<class 'list'>
24469


In [20]:
df2 = pd.DataFrame({'titles': ltitles, 'embeddings': lembeddings})
df2.head()

,titles,embeddings
0,Logic Programming in Assumption-Based Argument...,"[-0.023969512, 0.010221153, 0.0005422938, -0.0..."
1,Modeling and Mining Spatiotemporal Patterns of...,"[0.0034898794, -0.0044532777, -0.053476732, -0..."
2,Mitosis Detection in Breast Cancer Histology I...,"[0.013757976, -0.00069692935, 0.022236006, -0...."
3,Towards Population Scale Activity Recognition:...,"[0.06615189, 0.014459885, -0.035167716, -0.081..."
4,A Platform to Evaluate the Technology for Serv...,"[-0.0020297964, -0.03787091, -0.046355557, -0...."
